In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
LSTM_UNITS = 256
BATCH_SIZE = 100
LEARNING_RATE = 1e-3
N_CLASSES = 10
TRAINING_ITER = 1000

def LSTM(x):
    
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(num_units=LSTM_UNITS)
    
    init_state = lstm_cell.zero_state(batch_size=BATCH_SIZE, dtype=tf.float32)
    
    outputs, final_state = tf.nn.dynamic_rnn(cell=lstm_cell,inputs=x,initial_state=init_state,time_major=False) 
    
    return tf.unstack(tf.transpose(outputs, [1, 0, 2]))

def init_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

def init_biases(shape):
    return tf.Variable(tf.zeros(shape) + 0.1)

def add_layer(x, weights, biases, activation_function=None):
    results = tf.matmul(x, weights) + biases
    if activation_function:
        return activation_function(results)
    else:
        return results

weights = {'output': init_weights([LSTM_UNITS, N_CLASSES])}
biases = {'output': init_biases([N_CLASSES])}

x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, N_CLASSES])

x_2d = tf.reshape(x, [-1, 28, 28])

lstm_outputs = LSTM(x_2d)

predictions = add_layer(lstm_outputs[-1], weights['output'], biases['output'], None)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=predictions))

train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(loss)

accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.arg_max(y, 1), tf.arg_max(predictions, 1)), tf.float32))



In [4]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for i in range(TRAINING_ITER):
        batch_xs, batch_ys = mnist.train.next_batch(BATCH_SIZE)
        sess.run(train_step, feed_dict= {x: batch_xs, y: batch_ys})
        if i % 50 == 0:
            print 'TRAIN_ACC@%d:' %i, sess.run(accuracy, feed_dict = {x: batch_xs, y:batch_ys})
            
            
            #print 'VALID_ACC@%d:' %i, sess.run(accuracy, feed_dict = {x: mnist.validation.images, y: mnist.validation.labels})

TRAIN_ACC@0: 0.27
VALID_ACC@0:

InvalidArgumentError: ConcatOp : Dimensions of inputs should match: shape[0] = [5000,28] vs. shape[1] = [100,256]
	 [[Node: rnn/while/basic_lstm_cell/basic_lstm_cell/concat = ConcatV2[N=2, T=DT_FLOAT, Tidx=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"](rnn/while/TensorArrayReadV3, rnn/while/Identity_3, rnn/while/basic_lstm_cell/basic_lstm_cell/concat/axis)]]

Caused by op u'rnn/while/basic_lstm_cell/basic_lstm_cell/concat', defined at:
  File "/Users/miaofan/anaconda/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/miaofan/anaconda/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/miaofan/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/miaofan/anaconda/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/miaofan/anaconda/lib/